In [19]:
import tensorflow as tf

In [20]:
from __future__ import absolute_import, division, print_function, unicode_literals


import sys
import glob
import os
import time
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
from IPython.display import clear_output
from skimage.transform import pyramid_expand


from PIL import Image
import cv2


AUTOTUNE = tf.data.experimental.AUTOTUNE

In [21]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
BATCH_SIZE = 20

In [22]:
#이미지 주어진 크기에 따라 랜덤으로 분할,
def random_crop(image, c=3):
  cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, c])

  return cropped_image

In [23]:
# "-1 <= image <= 1" 로 변환
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return image

In [24]:
#이미지 변환 처리 중..
def random_jitter(image, c=3):
  # resizing to 286 x 286 x 3
  image = tf.image.resize(image, [256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  # randomly cropping to 256 x 256 x 3
  image = random_crop(image, c=c)

  # random mirroring
  #image = tf.image.random_flip_left_right(image)

  return image

In [25]:
def preprocess_image_train(image, label):
  image = random_jitter(image)
  image = normalize(image)
  return image

In [26]:
def preprocess_image_test(image, label):
  image = normalize(image)
  return image

In [27]:
def preprocess_image_test_nl(image):
  image = normalize(image)
  return image

In [28]:
def preprocess_image_train_nl(image, c=3):
  image = random_jitter(image, c=c)
  image = normalize(image)
  return image

In [29]:
def similar(n1, n2, d=0):
    sn = abs(n1-n2)
    if sn >= 0 and sn <= d :
        return True
    else :
        return False

In [77]:
class imagedata:
  def __init__(self, path,  w = 128, h = 192, degree=0):
    self.degree=degree
    self.outp = Image.open(path).convert("RGBA")
    
    
    
    if self.outp.height==192 and self.outp.width==128:
        pix = np.array(self.outp)[0][0]
        img = self.outp
        datas = img.getdata()
        newData = []
        for item in datas:
            if similar(item[0], pix[0], d=self.degree) and similar(item[1], pix[1], d=self.degree) and similar(item[2], pix[2], d=self.degree):
                newData.append((255, 255, 255, 0))
            else:
                newData.append(item)
        img.putdata(newData)
        self.outp = img
        
        self.outp = img.resize((w, h))
        area = (0, 0, int(w/4), int(h/4))
        self.inp = self.outp.crop(area)

        inp1 = np.array(self.inp)
        inp1 = cv2.cvtColor(inp1, cv2.COLOR_RGB2RGBA)
        inp2 = cv2.vconcat([inp1, inp1, inp1, inp1])
        self.inp = cv2.hconcat([inp2, inp2, inp2, inp2]) 

        self.outp = np.array(img)
        self.outp = cv2.cvtColor(self.outp, cv2.COLOR_RGB2RGBA)

       # print("인풋 사이즈 : " + str(self.inp.shape))
        #print("아웃풋 사이즈 : " + str(self.outp.shape))

        self.inp = preprocess_image_train_nl(self.inp, c=4)
        self.outp = preprocess_image_train_nl(self.outp, c=4)
        self.error = False
    else :
        self.inp = None
        self.outp = None
        self.error = True
    
    

In [65]:
'''PATH_DIR='./Dataset/'
filelist = []

for i in [file for file in os.listdir(PATH_DIR) if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpge") or file.endswith(".bmp")]:
  try :
    filelist.append(imagedata(PATH_DIR + str(i)))
  except :
    print(str(i)+" 파일 로드 실패")
    continue
  print(str(i)+" 파일 로드 완료")'''

'PATH_DIR=\'./Dataset/\'\nfilelist = []\n\nfor i in [file for file in os.listdir(PATH_DIR) if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpge") or file.endswith(".bmp")]:\n  try :\n    filelist.append(imagedata(PATH_DIR + str(i)))\n  except :\n    print(str(i)+" 파일 로드 실패")\n    continue\n  print(str(i)+" 파일 로드 완료")'

In [66]:
PATH_DIR='./Dataset/'
filelist = [file for file in os.listdir(PATH_DIR) if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpge") or file.endswith(".bmp")]

In [67]:
from random import sample
import random
def sampling_list(a, b, batch):
  batches = [random.randint(0,len(a)-1) for r in range(0, batch)]
  c = []
  d = []
  for j in batches:
    c.append(a[j])
    d.append(b[j])
  g = np.empty(shape=[0, IMG_HEIGHT, IMG_WIDTH, 4], dtype='float32')
  h = np.empty(shape=[0, IMG_HEIGHT, IMG_WIDTH, 4], dtype='float32')
  for i in range(0, batch):
    g = np.append(g, c[i][np.newaxis], axis=0)
    h = np.append(h, d[i][np.newaxis], axis=0)
  return g, h

In [74]:
from random import sample
#filelist.append(imagedata(PATH_DIR + str(i)))
def sampling_batch(fl, b, pd='./Dataset/'):
    #while True :
        #try :
            
            g = np.empty(shape=[0, IMG_HEIGHT, IMG_WIDTH, 4], dtype='float32')
            h = np.empty(shape=[0, IMG_HEIGHT, IMG_WIDTH, 4], dtype='float32')
            for i in sample(fl,b):
                if imagedata(pd+str(i)).error == True:
                    return None, None
                g = np.append(g, imagedata(pd+str(i)).inp[np.newaxis], axis=0)
                h = np.append(h, imagedata(pd+str(i)).outp[np.newaxis], axis=0)
            return g, h
       #except :
            #continue

In [79]:
def fileremove(file):
    if os.path.isfile(file):
        os.remove(file)
        return True
    else:
        return False

In [91]:
def imagefilter(path, num=0,  pd='./Dataset/') :
    PATH_DIR="./{0}/".format(path)
    filelist = [file for file in os.listdir(PATH_DIR) if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpge") or file.endswith(".bmp")]
    
    for i in range(num, len(filelist)): 
        try: 
            a, b= sampling_batch([filelist[i]], 1)
        except: 
            fileremove(pd+filelist[i])
            print(str(i)+"번째 이미지는 구성상 문제가 존재하여 삭제합니다.")

        try: 
            img = cv2.hconcat([(a[0]*0.5+0.5)*255, (b[0]*0.5+0.5)*255])
            print(str(i)+"번째 이미지는 정상입니다.")
        except:
            fileremove(pd+filelist[i])
            print(str(i)+"번째 이미지는 4x4 규격이 아닙니다.")

In [92]:
def imagesave(path, num=0) :
    PATH_DIR="./{0}/".format(path)
    filelist = [file for file in os.listdir(PATH_DIR) if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpge") or file.endswith(".bmp")]
    
    for i in range(num, len(filelist)): 
        try :
            a, b= sampling_batch([filelist[i]], 1)
        except: 
            continue

        try: 
            cv2.imwrite('{0}_combineImage/{1}.png'.format(path, i), cv2.hconcat([(a[0]*0.5+0.5)*255, (b[0]*0.5+0.5)*255]))
            print(str(i)+"번째 저장 완료")
        except:
            print(str(i)+"번째 이미지는 4x4 규격 아님")

In [ ]:
imagefilter("Dataset", num=)
print("train 필터링 완료")

24672번째 이미지는 정상입니다.
24673번째 이미지는 정상입니다.
24674번째 이미지는 정상입니다.
24675번째 이미지는 정상입니다.
24676번째 이미지는 정상입니다.
24677번째 이미지는 정상입니다.
24678번째 이미지는 정상입니다.
24679번째 이미지는 정상입니다.
24680번째 이미지는 정상입니다.
24681번째 이미지는 정상입니다.
24682번째 이미지는 정상입니다.
24683번째 이미지는 정상입니다.
24684번째 이미지는 정상입니다.
24685번째 이미지는 정상입니다.
24686번째 이미지는 정상입니다.
24687번째 이미지는 정상입니다.
24688번째 이미지는 정상입니다.
24689번째 이미지는 정상입니다.
24690번째 이미지는 정상입니다.
24691번째 이미지는 정상입니다.
24692번째 이미지는 정상입니다.
24693번째 이미지는 정상입니다.
24694번째 이미지는 정상입니다.
24695번째 이미지는 정상입니다.
24696번째 이미지는 정상입니다.
24697번째 이미지는 정상입니다.
24698번째 이미지는 정상입니다.
24699번째 이미지는 정상입니다.
24700번째 이미지는 정상입니다.
24701번째 이미지는 정상입니다.
24702번째 이미지는 정상입니다.
24703번째 이미지는 정상입니다.
24704번째 이미지는 정상입니다.
24705번째 이미지는 정상입니다.
24706번째 이미지는 정상입니다.
24707번째 이미지는 정상입니다.
24708번째 이미지는 정상입니다.
24709번째 이미지는 정상입니다.
24710번째 이미지는 정상입니다.
24711번째 이미지는 정상입니다.
24712번째 이미지는 정상입니다.
24713번째 이미지는 정상입니다.
24714번째 이미지는 정상입니다.
24715번째 이미지는 정상입니다.
24716번째 이미지는 정상입니다.
24717번째 이미지는 정상입니다.
24718번째 이미지는 정상입니다.
24719번째 이미지는 정상입니다.
24720번째 이미지는 정상입니다.
24721번째 이미지는 정상입니다.


In [ ]:
imagefilter("testinput")
print("test 필터링 완료")

In [ ]:
imagefilter("testinput2")
print("val 필터링 완료")

In [ ]:
imagesave("Dataset")
print("train save 완료")

In [ ]:
imagesave("testinput")
print("test save 완료")

In [ ]:
imagesave("testinput2")
print("val save 완료")